# ANALYZE EXPERIMENTS

In [1]:
import smarte.constants as cn
import smarte as smt

from dask.distributed import Client
import dask.dataframe as dd
import dask
import os

In [2]:
client = Client(n_workers=4)
client

Client Scheduler: tcp://127.0.0.1:44873 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 8.21 GB


# Create the dataframe of experiments

In [3]:
DF = dd.read_csv(os.path.join(cn.EXPERIMENT_DIR, "*.csv"))
DF

,biomodel_num,num_species,num_reaction,num_parameter,method,noise_mag,range_min_frac,range_max_frac,range_initial_frac,columns_deleted,max_fev,ts_instance,median_err,max_err,min_err,avg_time,cnt,tot_time,status
npartitions=7,,,,,,,,,,,,,,,,,,,
,int64,float64,float64,float64,object,float64,float64,int64,float64,int64,int64,int64,float64,float64,float64,float64,float64,float64,object
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [4]:
path = os.path.join(cn.EXPERIMENT_DIR, "workunits.txt")
with open(path, "r") as fd:
    workunit_strs = fd.readlines()
workunit_strs = [w if w[-1] == "l" else w[:-1] for w in workunit_strs]
workunits = [smt.Workunit.getFromStr(w) for w in workunit_strs]

In [5]:
# How many experiments were completed
len(DF)

16019

# Missing Experiments

In [6]:
set(["--".join(w.calcMultivaluedFactors()) for w in workunits])

{'biomodel_num--ts_instance'}

In [7]:
# Find what varies across workunits

In [8]:
# Combine workunits
agg_workunits = smt.ExtendedDict({k: [] for k in cn.SD_CONDITIONS})
for workunit in workunits[1:]:
    dct = workunit.removeExpansions()
    for key in dct.keys():
        # Handle bug where get [["differential_evolution"]]
        if isinstance(dct[key], str):
            values = set(agg_workunits[key]).union(dct[key])
        elif isinstance(dct[key], list):
            values = set(agg_workunits[key]).union(dct[key][0])
        agg_workunits[key] = list(values)
    print(agg_workunits)
if False:
    # Remove those known to vary
    for factor in [cn.SD_BIOMODEL_NUM, cn.SD_TS_INSTANCE]:
        del agg_workunits[factor]
    str(agg_workunits)

biomodel_num--l--a__method--differential_evolution__noise_mag--0.2__range_min_frac--0.5__range_max_frac--2__range_initial_frac--0.5__columns_deleted--0__max_fev--10000__ts_instance--l--a
biomodel_num--l--a__method--differential_evolution__noise_mag--0.2--0.1__range_min_frac--0.5__range_max_frac--2__range_initial_frac--0.5--0.8__columns_deleted--0__max_fev--10000__ts_instance--l--a
biomodel_num--l--a__method--differential_evolution__noise_mag--0.2--0.1__range_min_frac--0.5__range_max_frac--2__range_initial_frac--0.5--0.8__columns_deleted--0__max_fev--10000__ts_instance--l--a
biomodel_num--l--a__method--differential_evolution__noise_mag--0.2--0.1__range_min_frac--0.5__range_max_frac--2__range_initial_frac--0.5--0.8--1.2__columns_deleted--0__max_fev--10000__ts_instance--l--a
biomodel_num--l--a__method--differential_evolution__noise_mag--0.2--0.1__range_min_frac--0.5__range_max_frac--2__range_initial_frac--0.5--0.8--1.2__columns_deleted--0__max_fev--10000__ts_instance--l--a
biomodel_num--l

In [9]:
count_df = DF.groupby([cn.SD_NOISE_MAG, cn.SD_RANGE_INITIAL_FRAC, cn.SD_METHOD, cn.SD_MAX_FEV]).count()
df = count_df.compute()
ser = df[cn.SD_BIOMODEL_NUM]
ser

noise_mag  range_initial_frac  method                  max_fev
0.1        0.8                 differential_evolution  10000      3435
           1.2                 differential_evolution  10000      3435
           2.0                 differential_evolution  10000      3435
0.2        0.5                 differential_evolution  10000       725
           1.2                 differential_evolution  10000      2017
0.1        1.5                 leastsq                 10000      1485
           2.0                 leastsq                 10000      1487
Name: biomodel_num, dtype: int64

In [10]:
dff = DF[[cn.SD_NOISE_MAG, cn.SD_RANGE_INITIAL_FRAC, cn.SD_METHOD]]
len(dff)

16019

In [11]:
len(dff.drop_duplicates())

7